In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mayo-clinic-strip-ai/sample_submission.csv
/kaggle/input/mayo-clinic-strip-ai/train.csv
/kaggle/input/mayo-clinic-strip-ai/test.csv
/kaggle/input/mayo-clinic-strip-ai/other.csv
/kaggle/input/mayo-clinic-strip-ai/other/314263_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/0b33d7_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/54334d_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/84c279_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/cca9eb_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/880188_1.tif
/kaggle/input/mayo-clinic-strip-ai/other/2e3078_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/9daa03_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/065053_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/8acdfa_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/91530f_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/a2253f_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/f5eb8f_1.tif
/kaggle/input/mayo-clinic-strip-ai/other/ab830f_0.tif
/kaggle/input/mayo-clinic-strip-ai/other/d17ad3_2.tif

In [4]:
!pip uninstall -y lightning-flash pytorch-lightning torchvision torch lightning-bolts

Found existing installation: pytorch-lightning 2.4.0
Uninstalling pytorch-lightning-2.4.0:
  Successfully uninstalled pytorch-lightning-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0


In [ ]:
!pip install torch==1.13.1 torchvision==0.14.1
!pip install lightning-flash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 14.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 81.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 1.5 MB/s eta 0:00:00:00:0100:01


In [ ]:
import os
import glob
import pandas as pd
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
# 设置数据集路径
DATASET_FOLDER = "/kaggle/input/mayo-clinic-strip-ai/"
DATASET_SMALL_FOLDER = "/kaggle/input/newtrain"

# 验证路径是否存在
if not os.path.exists(DATASET_FOLDER):
    print(f"警告: {DATASET_FOLDER} 不存在")

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import keras.backend as K #to define custom loss function

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from pprint import pprint
from collections import defaultdict
import openslide
from openslide import OpenSlide

from glob import glob

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.layers import GlobalMaxPooling2D
from keras.models import load_model

print(keras.__version__)

In [ ]:
import random

train_df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')
test_df  = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')

# Specify patient_ids to remove
patient_ids_to_remove = ['006388', '008e5c', '00c058', '01adc5']

# Filter out the rows with specified patient_ids
train_df = train_df[~train_df['patient_id'].isin(patient_ids_to_remove)].reset_index(drop=True)
train_df = train_df.drop_duplicates(subset=['patient_id'])

df1 = train_df[train_df['label'] == 'CE']
df2 = train_df[train_df['label'] == 'LAA']
#adjust n to change number of CE data
sampled= df1.sample(n=200, random_state=42)
train_df = pd.concat([sampled, df2],ignore_index=True)
# Print the cleaned DataFrame
print("Cleaned DataFrame:")
print(train_df.head())
train_df['label'].value_counts()

In [ ]:
train_df['patient_id'].nunique

In [ ]:

train_images = glob("/kaggle/input/mayo-clinic-strip-ai/train/*")
test_images = glob("/kaggle/input/mayo-clinic-strip-ai/test/*")
other_images = glob("/kaggle/input/mayo-clinic-strip-ai/other/*")
print(f"Number of images in a training set: {len(train_images)}")
print(f"Number of images in a training set: {len(test_images)}")
print(f"Number of other: {len(other_images)}")

# Filtering out images based on the cleaned patient_ids in train_df
images_to_remove = [
    '/kaggle/input/mayo-clinic-strip-ai/train/006388_0.tif',
    '/kaggle/input/mayo-clinic-strip-ai/train/008e5c_0.tif',
    '/kaggle/input/mayo-clinic-strip-ai/train/00c058_0.tif',
    '/kaggle/input/mayo-clinic-strip-ai/train/01adc5_0.tif',
]

# Remove images associated with the patient_ids
train_images = [img for img in train_images if img not in images_to_remove]

# Check the total number of images after deletion
total_images_after_deletion = len(train_images)
print("Total number of images after deletion:", total_images_after_deletion)

# Print the paths of the cleaned list of images
print("First 5 image paths after deletion:")
print(train_images[:5])


In [ ]:
train_df["file_path"] = train_df["image_id"].apply(lambda x: "../input/mayo-clinic-strip-ai/train/" + x + ".tif")
test_df["file_path"]  = test_df["image_id"].apply(lambda x: "../input/mayo-clinic-strip-ai/test/" + x + ".tif")

In [ ]:
# labelling CE class as 1 and LAA as 0
train_df["target"] = train_df["label"].apply(lambda x : 1 if x=="CE" else 0)
train_df.head()

In [ ]:
%%time
train_df["file_path"] = train_df["image_id"].apply(lambda x: "../input/mayo-clinic-strip-ai/train/" + x + ".tif")

def preprocess(image_path):
    slide=OpenSlide(image_path)
    region= (2500,2500)    
    size  = (5000, 5000)
    image = slide.read_region(region, 0, size)
    image = image.resize((128, 128))
    image = np.array(image)    
    return image

X_train=[]
for i in tqdm(train_df['file_path']):
    x1=preprocess(i)
    X_train.append(x1)

Y_train=[]    
Y_train=train_df['target']

In [ ]:
X_train=np.array(X_train)
X_train=X_train/255.0
Y_train = np.array(Y_train)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X_train,Y_train, test_size=0.3, random_state=42)
X_train,X_val,Y_train,Y_val=train_test_split(X_train,Y_train, test_size=0.5, random_state=42)

In [ ]:
!pip install tensorflow numpy pandas tqdm


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
print("Y_train shape:", Y_train.shape)



In [ ]:
print("Unique values in Y_train:", np.unique(Y_train))


In [ ]:
from tensorflow.keras.utils import to_categorical

# 将整数标签转换为 one-hot 编码
Y_train = to_categorical(Y_train, num_classes=2)

print("Y_train shape after one-hot encoding:", Y_train.shape)
# 输出: (373, 2)


In [ ]:

from tensorflow.keras.utils import to_categorical



# 数据增强
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 生成增强数据
train_generator = datagen.flow(
    X_train, 
    Y_train, 
    batch_size=32  # 去掉 class_mode
)



In [ ]:
from tensorflow.keras.applications import EfficientNetB6
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical



# 加载 EfficientNet-B6 的预训练模型
base_model = EfficientNetB6(weights=None, include_top=False, input_shape=(128, 128, 4))

# 冻结预训练模型的卷积层
base_model.trainable = False

# 添加自定义分类头
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(Y_train.shape[1], activation='softmax')(x)  # 输出层

# 定义完整模型
model = Model(inputs=base_model.input, outputs=predictions)

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 打印模型结构
model.summary()



In [ ]:
import os
import subprocess
import tensorflow as tf

def check_cuda_version():
    try:
        # Run nvcc command to get CUDA version
        result = subprocess.run(['nvcc', '--version'], stdout=subprocess.PIPE, text=True)
        print("Current CUDA version:\n", result.stdout)
    except FileNotFoundError:
        print("CUDA not found. Please ensure it is installed.")

def check_cudnn_version():
    try:
        with open('/usr/include/cudnn_version.h', 'r') as f:
            for line in f:
                if 'CUDNN_MAJOR' in line or 'CUDNN_MINOR' in line or 'CUDNN_PATCHLEVEL' in line:
                    print(line.strip())
    except FileNotFoundError:
        print("CuDNN not found. Please ensure it is installed.")

def check_tf_gpu():
    print("Verifying TensorFlow GPU access...")
    gpus = tf.config.list_physical_devices('GPU')
    print("Num GPUs Available: ", len(gpus))

if __name__ == "__main__":
    check_cuda_version()
    check_cudnn_version()
    check_tf_gpu()


In [ ]:
conda install -c conda-forge cudnn=8.6.x  # Specify the version compatible with your TensorFlow


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 定义回调函数
early_stopping = EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)  # 监控训练集的 loss
model_checkpoint = ModelCheckpoint('efficientnet_b6_best_model.keras', save_best_only=True, monitor='loss')  # 保存基于训练集 loss 的最优模型

# 开始训练
history = model.fit(
    train_generator,  # 训练数据生成器
    epochs=20,        # 训练轮数
    callbacks=[early_stopping, model_checkpoint]
)


In [ ]:

test1=[]
for i in test_df['file_path']:
    x1=preprocess(i)
    test1.append(x1)
    print(i)
    
test1=np.array(test1)

In [ ]:
# 假设类别索引与名称的映射
class_names = ['CE', 'LAA']  # 0 对应 'CE'，1 对应 'LAA'

# 加载保存的模型
from tensorflow.keras.models import load_model
model = load_model('efficientnet_b6_best_model.keras')
print("Model loaded successfully!")

# 确保 test1 数据已经准备好并归一化（如果需要）
test1 = test1 / 255.0  # 如果像素值范围是 0-255，进行归一化



# 获取每张图片的预测类别索引
predicted_classes = predictions.argmax(axis=1)  # 获取每张图片的预测类别索引

# 打印每张图片的预测结果
for i, predicted_class in enumerate(predicted_classes):
    print(f"Image {i+1}: Predicted Class = {class_names[predicted_class]}")

# 可视化这 4 张图片及其预测结果
import matplotlib.pyplot as plt

for i in range(len(selected_images)):
    plt.imshow(selected_images[i])  # 显示图片
    plt.title(f"Predicted: {class_names[predicted_classes[i]]}")  # 显示预测类别
    plt.axis('off')
    plt.show()


In [ ]:
# 假设类别索引与名称的映射
class_names = ['CE', 'LAA']  # 0 对应 'CE'，1 对应 'LAA'

# 加载保存的模型
from tensorflow.keras.models import load_model
model = load_model('efficientnet_b6_best_model.keras')
print("Model loaded successfully!")

# 确保 test1 数据已经准备好并归一化（如果需要）
test1 = test1 / 255.0  # 如果像素值范围是 0-255，进行归一化

# 获取每张图片的预测结果
predictions = model.predict(test1)  # Perform prediction
predicted_classes = predictions.argmax(axis=1)  # 获取每张图片的预测类别索引

# 打印每张图片的预测结果
for i, predicted_class in enumerate(predicted_classes):
    print(f"Image {i + 1}: Predicted Class = {class_names[predicted_class]}")

# 可视化这 4 张图片及其预测结果
import matplotlib.pyplot as plt

# 选择要可视化的图片，假设 selected_images 是你之前定义的变量
for i in range(len(selected_images)):
    plt.imshow(selected_images[i])  # 显示图片
    plt.title(f"Predicted: {class_names[predicted_classes[i]]}")  # 显示预测类别
    plt.axis('off')  # 不显示坐标轴
    plt.show()  # 显示图片


In [ ]:
print(type(Y_train))  # 检查数据类型
print(Y_train.shape)  # 检查形状
